<div style="text-align: center;">
    <p style="background-color: #8aeb92; color: #910909; font-size: 2.0em; font-family: 'Arial', sans-serif; text-align: center; border-radius: 25px; border: 4px solid #910909; display: inline-block; padding: 20px 20px; margin: 0 auto; line-height: 1.5; font-weight: bold; box-shadow: 0px 4px 10px rgba(0, 0, 0, 0.3);">
        🐱🐶 Cat or Dog? Binary Classification using Deep Learning 🧠📊
    </p>
</div>


<div style="text-align: center;">
    <p style="background-color: #8aeb92; color: #910909; font-size: 2.0em; font-family: 'Arial', sans-serif; text-align: center; border-radius: 25px; border: 4px solid #910909; display: inline-block; padding: 20px 20px; margin: 0 auto; line-height: 1.5; font-weight: bold; box-shadow: 0px 4px 10px rgba(0, 0, 0, 0.3);">
        📊 About the Dataset 🐱🐶
    </p>
    <p style="font-size: 1.0em; font-family: 'Arial', sans-serif; text-align: justify; line-height: 1.5; padding: 10px;">
        This dataset contains two classes of images: <a href = "https://www.kaggle.com/datasets/shaunthesheep/microsoft-catsvsdogs-dataset"><b>Cats</b> and <b>Dogs</b></a>. It is well-suited for binary classification tasks in computer vision, where the objective is to build a model that can differentiate between these two categories. Each image in the dataset is labeled accordingly, making it an excellent resource for training, validating, and testing deep learning models focused on binary image classification.
    </p>
</div>


<div style="text-align: center;">
    <p style="background-color: #8aeb92; color: #910909; font-size: 2.0em; font-family: 'Arial', sans-serif; text-align: center; border-radius: 25px; border: 4px solid #910909; display: inline-block; padding: 20px 20px; margin: 0 auto; line-height: 1.5; font-weight: bold; box-shadow: 0px 4px 10px rgba(0, 0, 0, 0.3);">
        📊 Import Libraries 🐱🐶
    </p>
</div>


In [ ]:
import os
import PIL
import glob
import shutil
import random
import pandas as pd
from PIL import Image
import tensorflow as tf
import tensorflow_hub as hub
import tf_keras as keras
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, Add, MaxPooling2D, Dense, BatchNormalization, Dropout, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import ResNet50, VGG16, MobileNetV2

<div style="text-align: center;">
    <p style="background-color: #8aeb92; color: #910909; font-size: 2.0em; font-family: 'Arial', sans-serif; text-align: center; border-radius: 25px; border: 4px solid #910909; display: inline-block; padding: 20px 20px; margin: 0 auto; line-height: 1.5; font-weight: bold; box-shadow: 0px 4px 10px rgba(0, 0, 0, 0.3);">
        📂 Loading
    </p>
    <p style="font-size: 1.0em; font-family: 'Arial', sans-serif; text-align: justify; line-height: 1.5; padding: 10px;">
        In this section, we import the necessary libraries and specify the path to our dataset. Using the <code>os</code> library, we access the dataset directory and automatically identify the class names from the folder structure. This allows us to confirm the two categories, Cats and Dogs, which will be used for binary classification. Displaying the class names helps verify the dataset setup before proceeding with further processing.
    </p>
</div>


### <b><i><center> Cruppted Image Remover

In [ ]:
corrupted_images = []

def is_valid_image(image_path):
    try:
        img = Image.open(image_path)
        img.verify()
        return True
    except (IOError, SyntaxError, PIL.UnidentifiedImageError):
        print(f"Corrupted image: {image_path}")
        corrupted_images.append(image_path)
        return False

<div style="text-align: center;">
    <p style="background-color: #8aeb92; color: #910909; font-size: 2.0em; font-family: 'Arial', sans-serif; text-align: center; border-radius: 25px; border: 4px solid #910909; display: inline-block; padding: 20px 20px; margin: 0 auto; line-height: 1.5; font-weight: bold; box-shadow: 0px 4px 10px rgba(0, 0, 0, 0.3);">
        📊 Loading Data From <b>KAGGLE</b> 🐱🐶
    </p>
</div>


In [ ]:
import kagglehub
path = kagglehub.dataset_download("shaunthesheep/microsoft-catsvsdogs-dataset")

print("Path to dataset files:", path)

100%|██████████| 788M/788M [00:06<00:00, 132MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/shaunthesheep/microsoft-catsvsdogs-dataset/versions/1


In [ ]:
print("Path to dataset files:", path)
print("Contents of dataset directory:", os.listdir(path))

Path to dataset files: /root/.cache/kagglehub/datasets/shaunthesheep/microsoft-catsvsdogs-dataset/versions/1
Contents of dataset directory: ['PetImages', 'MSR-LA - 3467.docx', 'readme[1].txt']


In [ ]:
data_dir = os.path.join(path, 'PetImages')

In [ ]:
output_dir = '/content/data'

In [ ]:
train_dir = os.path.join(output_dir, 'train')
val_dir = os.path.join(output_dir, 'val')
test_dir = os.path.join(output_dir, 'test')

In [ ]:
for folder in [train_dir, val_dir, test_dir]:
    os.makedirs(os.path.join(folder, 'cats'), exist_ok=True)
    os.makedirs(os.path.join(folder, 'dogs'), exist_ok=True)

In [ ]:
cat_images = glob.glob(os.path.join(data_dir, 'Cat', '*'))
dog_images = glob.glob(os.path.join(data_dir, 'Dog', '*'))

In [ ]:
cat_images = [img for img in cat_images if is_valid_image(img)]
dog_images = [img for img in dog_images if is_valid_image(img)]

Corrupted image: /root/.cache/kagglehub/datasets/shaunthesheep/microsoft-catsvsdogs-dataset/versions/1/PetImages/Cat/666.jpg
Corrupted image: /root/.cache/kagglehub/datasets/shaunthesheep/microsoft-catsvsdogs-dataset/versions/1/PetImages/Cat/Thumbs.db
Corrupted image: /root/.cache/kagglehub/datasets/shaunthesheep/microsoft-catsvsdogs-dataset/versions/1/PetImages/Dog/Thumbs.db


/usr/local/lib/python3.10/dist-packages/PIL/TiffImagePlugin.py:935: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Corrupted image: /root/.cache/kagglehub/datasets/shaunthesheep/microsoft-catsvsdogs-dataset/versions/1/PetImages/Dog/11702.jpg


In [ ]:
random.shuffle(cat_images)
random.shuffle(dog_images)

<div style="text-align: center;">
    <p style="background-color: #8aeb92; color: #910909; font-size: 2.0em; font-family: 'Arial', sans-serif; text-align: center; border-radius: 25px; border: 4px solid #910909; display: inline-block; padding: 20px 20px; margin: 0 auto; line-height: 1.5; font-weight: bold; box-shadow: 0px 4px 10px rgba(0, 0, 0, 0.3);">
        📂 Creating and Splitting Data into Train, Validation and Test Sets
    </p>
</div>


In [ ]:
train_cats, val_cats, test_cats = cat_images[:11251], cat_images[11251:11876], cat_images[11876:]
train_dogs, val_dogs, test_dogs = dog_images[:11251], dog_images[11251:11876], dog_images[11876:]

### <b><i><center> Moving Images to Folders

In [ ]:
for img in train_cats:
    shutil.copy(img, os.path.join(train_dir, 'cats'))
for img in val_cats:
    shutil.copy(img, os.path.join(val_dir, 'cats'))
for img in test_cats:
    shutil.copy(img, os.path.join(test_dir, 'cats'))

for img in train_dogs:
    shutil.copy(img, os.path.join(train_dir, 'dogs'))
for img in val_dogs:
    shutil.copy(img, os.path.join(val_dir, 'dogs'))
for img in test_dogs:
    shutil.copy(img, os.path.join(test_dir, 'dogs'))

print("Data split and stored in '/content/data' directory")

Data split and stored in '/content/data' directory


In [ ]:
base_dir = '/content/data'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')

<div style="text-align: center;">
    <p style="background-color: #8aeb92; color: #910909; font-size: 2.0em; font-family: 'Arial', sans-serif; text-align: center; border-radius: 25px; border: 4px solid #910909; display: inline-block; padding: 20px 20px; margin: 0 auto; line-height: 1.5; font-weight: bold; box-shadow: 0px 4px 10px rgba(0, 0, 0, 0.3);">
        🗂️ Preparing Data Generators for Training, Validation and Testing
    </p>
    <p style="font-size: 1.0em; font-family: 'Arial', sans-serif; text-align: justify; line-height: 1.5; padding: 10px;">
        In this step, we set up data generators to feed images into the model for training and validation. Using TensorFlow's <code>ImageDataGenerator</code>, we preprocess the images by rescaling their pixel values. First, we clean the training and testing directories by removing any invalid images, ensuring a smooth data flow. The data generators are then configured to load images from these directories, resize them to 224x224 pixels, and set a batch size of 32. This setup streamlines the data pipeline, preparing it for efficient model training and evaluation.
    </p>
</div>


In [ ]:
train_gen = ImageDataGenerator(rescale = 1./255,
                               shear_range = 0.1,
                               zoom_range = 0.2,
                               horizontal_flip = True,
                               width_shift_range = 0.1,
                               height_shift_range = 0.1)
val_gen = ImageDataGenerator(rescale = 1./255)
test_gen = ImageDataGenerator(rescale = 1./255)

In [ ]:
train_generator = train_gen.flow_from_directory(
        '/content/data/train',
        target_size = (224, 224),
        batch_size = 128,
        class_mode = 'binary')

val_generator = val_gen.flow_from_directory(
        '/content/data/val',
        target_size = (224, 224),
        batch_size = 128,
        class_mode = 'binary')

test_generator = test_gen.flow_from_directory(
        '/content/data/test',
        target_size = (224, 224),
        batch_size = 128,
        class_mode = 'binary')

Found 22502 images belonging to 2 classes.
Found 1250 images belonging to 2 classes.
Found 1246 images belonging to 2 classes.


<div style="text-align: center;">
    <p style="background-color: #8aeb92; color: #910909; font-size: 2.0em; font-family: 'Arial', sans-serif; text-align: center; border-radius: 25px; border: 4px solid #910909; display: inline-block; padding: 20px 20px; margin: 0 auto; line-height: 1.5; font-weight: bold; box-shadow: 0px 4px 10px rgba(0, 0, 0, 0.3);">
        🏗️ Building a CNN Model From Scratch
    </p>
</div>


In [ ]:
model = tf.keras.models.Sequential()

# first layer
model.add(Conv2D(32, (3,3), activation = 'relu', input_shape = (224, 224, 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.2))

# 2nd layer
model.add(Conv2D(64, (3,3), activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.2))

# 3rd layer
model.add(Conv2D(128, (3,3), activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.2))


# 4tg layer
model.add(Conv2D(256, (3,3), activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.2))

# flattening
model.add(Flatten())

# fully connected
model.add(Dense(256, activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

# output layer
model.add(Dense(1, activation = 'sigmoid'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


<div style="text-align: center;">
    <p style="background-color: #8aeb92; color: #910909; font-size: 2.0em; font-family: 'Arial', sans-serif; text-align: center; border-radius: 25px; border: 4px solid #910909; display: inline-block; padding: 20px 20px; margin: 0 auto; line-height: 1.5; font-weight: bold; box-shadow: 0px 4px 10px rgba(0, 0, 0, 0.3);">
        🌐 Model Summary
    </p>
</div>


In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 222, 222, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 222, 222, 32)        │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 109, 109, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 109, 109, 64)        │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 52, 52, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 52, 52, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 26, 26, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 26, 26, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 24, 24, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 24, 24, 256)         │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 12, 12, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 12, 12, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 36864)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │       9,437,440 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │              

 Total params: 9,829,057 (37.49 MB)

 Trainable params: 9,827,585 (37.49 MB)

 Non-trainable params: 1,472 (5.75 KB)

<div style="text-align: center;">
    <p style="background-color: #8aeb92; color: #910909; font-size: 2.0em; font-family: 'Arial', sans-serif; text-align: center; border-radius: 25px; border: 4px solid #910909; display: inline-block; padding: 20px 20px; margin: 0 auto; line-height: 1.5; font-weight: bold; box-shadow: 0px 4px 10px rgba(0, 0, 0, 0.3);">
        ⚙️ Compiling Model
    </p>
</div>


In [ ]:
opt = tf.keras.optimizers.RMSprop(learning_rate=0.001)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

<div style="text-align: center;">
    <p style="background-color: #8aeb92; color: #910909; font-size: 2.0em; font-family: 'Arial', sans-serif; text-align: center; border-radius: 25px; border: 4px solid #910909; display: inline-block; padding: 20px 20px; margin: 0 auto; line-height: 1.5; font-weight: bold; box-shadow: 0px 4px 10px rgba(0, 0, 0, 0.3);">
        ⚙️ Training the Model
    </p>
</div>


In [ ]:
history = model.fit(train_generator, epochs=10, validation_data=val_generator)

<div style="text-align: center;">
    <p style="background-color: #8aeb92; color: #910909; font-size: 2.0em; font-family: 'Arial', sans-serif; text-align: center; border-radius: 25px; border: 4px solid #910909; display: inline-block; padding: 20px 20px; margin: 0 auto; line-height: 1.5; font-weight: bold; box-shadow: 0px 4px 10px rgba(0, 0, 0, 0.3);">
        📊 Evaluating Model Performance (Loss and Accuracy)
    </p>
</div>


In [ ]:
result = pd.DataFrame(history.history)
result.tail()

In [ ]:
plt.style.use('ggplot')
plt.figure(figsize=(10,5))
result[['loss', 'val_loss']].plot()
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss Over Epochs')
plt.show()

In [ ]:
plt.style.use('ggplot')
plt.figure(figsize=(10,5))
result[['accuracy', 'val_accuracy']].plot()
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Accuracy Over Epochs')
plt.show()

<div style="text-align: center;">
    <p style="background-color: #8aeb92; color: #910909; font-size: 2.0em; font-family: 'Arial', sans-serif; text-align: center; border-radius: 25px; border: 4px solid #910909; display: inline-block; padding: 20px 20px; margin: 0 auto; line-height: 1.5; font-weight: bold; box-shadow: 0px 4px 10px rgba(0, 0, 0, 0.3);">
        📊 Evaluating Model Performance on Test Data
    </p>
</div>


In [ ]:
evaluation = model.evaluate(test_generator)
print(f'Test Accuracy: {evaluation[1] * 100:.2f}%')

<div style="text-align: center;">
    <p style="background-color: #8aeb92; color: #910909; font-size: 2.0em; font-family: 'Arial', sans-serif; text-align: center; border-radius: 25px; border: 4px solid #910909; display: inline-block; padding: 20px 20px; margin: 0 auto; line-height: 1.5; font-weight: bold; box-shadow: 0px 4px 10px rgba(0, 0, 0, 0.3);">
        🏗️ Building a Transfer Learning Model
    </p>
    <p style="font-size: 1.0em; font-family: 'Arial', sans-serif; text-align: justify; line-height: 1.5; padding: 10px;">
        Here, we define a function to create a transfer learning model using TensorFlow Hub. This function, <code>create_model</code>, accepts a URL for a pre-trained model and the number of output classes. It builds a model with two main components: a pre-trained feature extraction layer that captures high-level patterns from the input images, and a dense output layer for binary classification. By freezing the feature extraction layer, we focus on training only the output layer, which helps speed up the training process and reduce the risk of overfitting. This approach efficiently adapts a powerful pre-trained model for our specific classification task.
    </p>
</div>


In [ ]:
resnet_url = "https://www.kaggle.com/models/tensorflow/resnet-50/TensorFlow2/classification/1"
efficientnet_url = "https://www.kaggle.com/models/google/efficientnet-v2/TensorFlow2/imagenet1k-b0-classification/2"
mobilenet_url = "https://kaggle.com/models/google/mobilenet-v2/TensorFlow2/100-224-feature-vector/1"

In [ ]:
IMAGE_SHAPE = (224,224)
def create_model(model_url, num_classes):

  feature_extractor_layer = hub.KerasLayer(model_url,
                                           trainable=False,
                                           name='feature_extraction_layer',
                                           input_shape=IMAGE_SHAPE+(3,))


  model = keras.Sequential([
    feature_extractor_layer,
    keras.layers.Dense(num_classes, activation='sigmoid', name='output_layer')

  ])
  return model

<div style="text-align: center;">
    <p style="background-color: #8aeb92; color: #910909; font-size: 2.0em; font-family: 'Arial', sans-serif; text-align: center; border-radius: 25px; border: 4px solid #910909; display: inline-block; padding: 20px 20px; margin: 0 auto; line-height: 1.5; font-weight: bold; box-shadow: 0px 4px 10px rgba(0, 0, 0, 0.3);">
        ⚙️ Transfer Learning with ResNet Model
    </p>
</div>


In [ ]:
resnet_model = create_model(resnet_url, num_classes=1)

resnet_model.compile(loss='binary_crossentropy',
                     optimizer=keras.optimizers.Adam(),
                     metrics=['accuracy'])

resnet_history = resnet_model.fit(train_generator,
                                  validation_data=val_generator,
                                  epochs=5,
                                  steps_per_epoch=len(train_generator),
                                  validation_steps=len(val_generator)
                                  )